In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("julian3833/jigsaw-toxic-comment-classification-challenge")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'jigsaw-toxic-comment-classification-challenge' dataset.
Path to dataset files: /kaggle/input/jigsaw-toxic-comment-classification-challenge


LOAD DATA

In [4]:
import pandas as pd
import numpy as np


Load Dataset

In [5]:
import os

file_path = os.path.join(path, "train.csv")
df = pd.read_csv(file_path, engine='python')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
df.columns


Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [7]:
df.shape


(159571, 8)

In [8]:
df['toxic'].value_counts()


,count
toxic,
0,144277
1,15294


In [9]:
X = df['comment_text']   # input
y = df['toxic']          # output


Import Required Libraries

In [10]:
import re #re = regular expressionsUsed for text cleaning


Create a Text Cleaning Function

In [11]:
def clean_text(text):
    text = text.lower()                     # lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text) # remove punctuation & numbers
    text = re.sub(r'\s+', ' ', text).strip()# remove extra spaces
    return text


Apply Cleaning to Dataset

In [12]:
df['clean_comment'] = df['comment_text'].apply(clean_text)
df[['comment_text', 'clean_comment']].head()


,comment_text,clean_comment
0,Explanation\nWhy the edits made under my usern...,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,daww he matches this background colour im seem...
2,"Hey man, I'm really not trying to edit war. It...",hey man im really not trying to edit war its j...
3,"""\nMore\nI can't make any real suggestions on ...",more i cant make any real suggestions on impro...
4,"You, sir, are my hero. Any chance you remember...",you sir are my hero any chance you remember wh...


Verify Cleaning Worked

In [13]:
df['clean_comment'].iloc[0]


'explanation why the edits made under my username hardcore metallica fan were reverted they werent vandalisms just closure on some gas after i voted at new york dolls fac and please dont remove the template from the talk page since im retired now'

Import TF-IDF Vectorizer


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
#This tool converts cleaned text into numerical features

#Widely used in industry for text classification

Initialize TF-IDF Vectorizer

In [15]:
tfidf = TfidfVectorizer(
    max_features=5000,
    stop_words='english'
)
#max_features=5000 → limits vocabulary size (controls complexity)

#stop_words='english' → removes useless words like the, is, and

Fit & Transform Text Data

In [16]:
X_tfidf = tfidf.fit_transform(df['clean_comment'])


Check Shape of TF-IDF Output

In [17]:
X_tfidf.shape


(159571, 5000)

Confirm Target Variable

In [18]:
y = df['toxic']


Import Train-Test Split Tool

In [19]:
from sklearn.model_selection import train_test_split


Perform the Split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf,
    y,
    test_size=0.2,
    random_state=42
)
#test_size=0.2 → 80% training, 20% testing

#random_state=42 → reproducibility

Verify Split Sizes

In [21]:
X_train.shape, X_test.shape


((127656, 5000), (31915, 5000))

TRAIN YOUR FIRST MODEL
Logistic Regression (Industry-Safe Choice)

Import the Model

In [22]:
from sklearn.linear_model import LogisticRegression


Initialize the Model

In [23]:
model = LogisticRegression(max_iter=1000)


Train the Model

In [24]:
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

Make Predictions

In [25]:
y_pred = model.predict(X_test)


MODEL EVALUATION

Import Evaluation Tools

In [26]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


Calculate Accuracy

In [27]:
accuracy = accuracy_score(y_test, y_pred)
accuracy


0.955976813410622

Detailed Classification Report

In [28]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.96      0.99      0.98     28859
           1       0.90      0.61      0.73      3056

    accuracy                           0.96     31915
   macro avg       0.93      0.80      0.85     31915
weighted avg       0.95      0.96      0.95     31915



Confusion Matrix

In [29]:
confusion_matrix(y_test, y_pred)


array([[28645,   214],
       [ 1191,  1865]])

In [30]:
import pandas as pd

y_prob = model.predict_proba(X_test)[:, 1]

results = pd.DataFrame({
    "comment": df.loc[y_test.index, "comment_text"],
    "true_label": y_test,
    "probability": y_prob
})

results["pred_label"] = (results["probability"] >= 0.5).astype(int)


In [31]:
false_positives = results[
    (results["true_label"] == 0) & (results["pred_label"] == 1)
]

false_positives.head(5)


,comment,true_label,probability,pred_label
144017,"""\n\nIt's OK folks, FreeRangeFrog is clearly m...",0,0.516598,1
3056,Ram it up ur ass very hard till ur eyes water,0,0.783158,1
149385,What the heck i have a family of 2000 that i m...,0,0.775787,1
48202,"Seriously \n\nSeriously dude, quit it, you're ...",0,0.533450,1
34137,Youre a homo for protectin Archbelwtfishisname...,0,0.530178,1


In [32]:
false_negatives = results[
    (results["true_label"] == 1) & (results["pred_label"] == 0)
]

false_negatives.head(5)


,comment,true_label,probability,pred_label
56047,"""\n\n Sockpuppet? \n\nOh no Stifle. Just what ...",1,0.098504,0
19915,""" You're a dumb American, right? No degree? Kn...",1,0.247302,0
718,Do not help the jew Schnider. Final warning.,1,0.127046,0
107574,Don't talk to me troll.,1,0.343210,0
39699,"""\nI nearly get blocked? Lol. I didn't broke a...",1,0.080923,0


TEST WITH CUSTOM INPUT

Create Prediction Function

In [33]:
def predict_multi_toxicity(text):
    clean = clean_text(text)
    vector = tfidf.transform([clean])

    preds = multi_model.predict(vector)[0]
    probs = multi_model.predict_proba(vector)[0]

    results = {}
    for label, pred, prob in zip(label_columns, preds, probs):
        results[label] = {
            "prediction": int(pred),
            "confidence": round(prob, 2)
        }
    return results


In [34]:
%%writefile app.py
import streamlit as st
import re
import pickle

# Load saved objects
model = pickle.load(open("model.pkl", "rb"))
tfidf = pickle.load(open("tfidf.pkl", "rb"))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

st.title("AI Content Moderation System")

user_input = st.text_area("Enter a comment to check toxicity:")

if st.button("Check Toxicity"):
    clean = clean_text(user_input)
    vector = tfidf.transform([clean])
    prediction = model.predict(vector)[0]
    probability = model.predict_proba(vector)[0][1]

    if prediction == 1:
        st.error(f"Toxic Content Detected 🚨 (Confidence: {probability:.2f})")
    else:
        st.success(f"Non-Toxic Content ✅ (Confidence: {1 - probability:.2f})")


Writing app.py


In [35]:
import pickle

pickle.dump(model, open("model.pkl", "wb"))
pickle.dump(tfidf, open("tfidf.pkl", "wb"))


In [36]:
!pip install streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 99.9 MB/s eta 0:00:00


In [37]:
from pyngrok import ngrok
ngrok.set_auth_token("36v8FeQSHcYsQGJf4GxyLztXt85_2KaKA6HbTFcU8sCqnG4zv")


In [38]:
import pickle

pickle.dump(model, open("model.pkl", "wb"))
pickle.dump(tfidf, open("tfidf.pkl", "wb"))


In [39]:
!ls


app.py	model.pkl  sample_data	tfidf.pkl


In [40]:
!streamlit run app.py &>/content/logs.txt &


In [41]:
public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://leery-unarticulately-detra.ngrok-free.dev" -> "http://localhost:8501">